文本特征处理

In [39]:

with  open('/home/its/CodeReview/LLM/data/kapok.txt', 'r', encoding='utf-8') as txt_file:
    #txt_list = []
    for i in txt_file:
        txt_list.append([j.strip() for j in i.split(',')])

SyntaxError: closing parenthesis ')' does not match opening parenthesis '[' (956036434.py, line 4)

In [38]:
txt_list

[['致橡树'],
 ['【作者】舒婷 【朝代】现代'],
 ['我如果爱你——'],
 ['绝不像攀援的凌霄花，'],
 ['借你的高枝炫耀自己；'],
 ['我如果爱你——'],
 ['绝不学痴情的鸟儿，'],
 ['为绿荫重复单调的歌曲；'],
 ['也不止像泉源，'],
 ['常年送来清凉的慰藉；'],
 ['也不止像险峰，'],
 ['增加你的高度，衬托你的威仪。'],
 ['甚至日光，'],
 ['甚至春雨。'],
 [''],
 ['不，这些都还不够！'],
 ['我必须是你近旁的一株木棉，'],
 ['作为树的形象和你站在一起。'],
 ['根，紧握在地下；'],
 ['叶，相触在云里。'],
 ['每一阵风过，'],
 ['我们都互相致意，'],
 ['但没有人，'],
 ['听懂我们的言语。'],
 ['你有你的铜枝铁干，'],
 ['像刀，像剑，也像戟；'],
 ['我有我红硕的花朵，'],
 ['像沉重的叹息，'],
 ['又像英勇的火炬。'],
 [''],
 ['我们分担寒潮、风雷、霹雳；'],
 ['我们共享雾霭、流岚、虹霓。'],
 ['仿佛永远分离，'],
 ['却又终身相依。'],
 ['这才是伟大的爱情，'],
 ['坚贞就在这里：'],
 ['爱——'],
 ['不仅爱你伟岸的身躯，'],
 ['也爱你坚持的位置，'],
 ['足下的土地。'],
 ['致橡树'],
 ['【作者】舒婷 【朝代】现代'],
 ['我如果爱你——'],
 ['绝不像攀援的凌霄花，'],
 ['借你的高枝炫耀自己；'],
 ['我如果爱你——'],
 ['绝不学痴情的鸟儿，'],
 ['为绿荫重复单调的歌曲；'],
 ['也不止像泉源，'],
 ['常年送来清凉的慰藉；'],
 ['也不止像险峰，'],
 ['增加你的高度，衬托你的威仪。'],
 ['甚至日光，'],
 ['甚至春雨。'],
 [''],
 ['不，这些都还不够！'],
 ['我必须是你近旁的一株木棉，'],
 ['作为树的形象和你站在一起。'],
 ['根，紧握在地下；'],
 ['叶，相触在云里。'],
 ['每一阵风过，'],
 ['我们都互相致意，'],
 ['但没有人，'],
 ['听懂我们的言语。'],
 ['你有你的铜枝铁干，'],
 ['像刀，像剑，也像戟；'],


In [18]:
raw_corpus = [
 '致橡树',
 '【作者】舒婷 【朝代】现代',
 '我如果爱你——',
 '绝不像攀援的凌霄花，',
 '借你的高枝炫耀自己；',
 '我如果爱你——',
 '绝不学痴情的鸟儿，',
 '为绿荫重复单调的歌曲；',
 '也不止像泉源，',
 '常年送来清凉的慰藉；',
 '也不止像险峰，',
 '增加你的高度，衬托你的威仪。',
 '甚至日光，',
 '甚至春雨。'
]

In [20]:
import re
import jieba
import jieba.analyse

jieba.enable_parallel(8)
#jieba.load_userdict('')
jieba.analyse.set_stop_words('/home/its/CodeReview/LLM/data/baidu_stopwords.txt')
for sentence in raw_corpus:
    sentence = ''.join(re.findall(r'[\u4e00-\u9fa5]+', sentence))
    corpus.append([item for item in jieba.cut(sentence)])

#[print([jieba.cut(word)]) for word in raw_corpus ]


In [21]:
corpus

[<generator object _pcut at 0x7fee9b909d40>,
 ['致橡树'],
 ['作者', '舒婷', '朝代', '现代'],
 ['我', '如果', '爱', '你'],
 ['绝不', '像', '攀援', '的', '凌霄花'],
 ['借', '你', '的', '高枝', '炫耀', '自己'],
 ['我', '如果', '爱', '你'],
 ['绝不', '学', '痴情', '的', '鸟儿'],
 ['为', '绿荫', '重复', '单调', '的', '歌曲'],
 ['也', '不止', '像', '泉源'],
 ['常年', '送来', '清凉', '的', '慰藉'],
 ['也', '不止', '像', '险峰'],
 ['增加', '你', '的', '高度', '衬托', '你', '的', '威仪'],
 ['甚至', '日光'],
 ['甚至', '春雨'],
 ['致橡树'],
 ['作者', '舒婷', '朝代', '现代'],
 ['我', '如果', '爱', '你'],
 ['绝不', '像', '攀援', '的', '凌霄花'],
 ['借', '你', '的', '高枝', '炫耀', '自己'],
 ['我', '如果', '爱', '你'],
 ['绝不', '学', '痴情', '的', '鸟儿'],
 ['为', '绿荫', '重复', '单调', '的', '歌曲'],
 ['也', '不止', '像', '泉源'],
 ['常年', '送来', '清凉', '的', '慰藉'],
 ['也', '不止', '像', '险峰'],
 ['增加', '你', '的', '高度', '衬托', '你', '的', '威仪'],
 ['甚至', '日光'],
 ['甚至', '春雨'],
 ['致橡树'],
 ['作者', '舒婷', '朝代', '现代'],
 ['我', '如果', '爱', '你'],
 ['绝不', '像', '攀援', '的', '凌霄花'],
 ['借', '你', '的', '高枝', '炫耀', '自己'],
 ['我', '如果', '爱', '你'],
 ['绝不', '学', '痴情', '的', '鸟儿'],
 ['为', '绿荫'

In [34]:
from gensim import corpora

dictionary = corpora.Dictionary(corpus)
dictionary.save('/home/its/CodeReview/LLM/data/love.dict')

In [35]:
dictionary.filter_extremes(no_below=20, no_above=0.5)  # 删掉只在不超过20个文本中出现过的词，删掉在50%及以上的文本都出现了的词
# dictionary.filter_tokens(['一个'])  # 这个函数可以直接删除指定的词
dictionary.compactify()  # 去掉因删除词汇而出现的空白

In [39]:
corpus = [dictionary.doc2bow(s) for s in corpus]
corpora.MmCorpus.serialize('corpus_bow.mm', corpus)  # 存储语料库

In [40]:
from gensim.models import LdaModel

temp = dictionary[0]
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=2000,
    alpha='auto',
    eta='auto',
    iterations=500,
    num_topics=3,
    passes=20,
    eval_every=None
)

# model.save('')

KeyError: 0

In [38]:
top_topics = model.top_topics(corpus) #, num_words=20)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

from pprint import pprint
pprint(top_topics)

NameError: name 'model' is not defined

In [ ]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(model, corpus, dictionary)
# 需要的三个参数都可以从硬盘读取的，前面已经存储下来了

pyLDAvis.show(vis)